In [55]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.llms.ollama import Ollama
from llama_index.core.workflow import Context
import asyncio
import os

In [56]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
Settings.llm = Ollama(
    model="llama3.2:latest",
    request_timeout=360.0,
    # Manually set the context window to limit memory usage
    context_window=8000,
)

In [57]:
documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [69]:
cart = []

# Define a simple calculator tool
def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    print("Using multiply tool")
    return a + b

async def search_documents(query: str) -> str:
    """Useful for answering natural language questions about products of a furniture store items."""
    response = await query_engine.aquery(query)
    return str(response)

def addToCart(item)->str:
    """Useful for adding an item to the cart"""
    cart.append(item)
    return "Item added to the cart"

# Create an agent workflow with our calculator tool
agent = FunctionAgent(
    tools=[multiply,addToCart,search_documents],
    llm=Ollama(model="llama3.2:latest",thinking=False),
    system_prompt="""You are a helpful assistant that can : 
    1. multiply two numbers,
    2. Add an item to the cart.
    """,
)

In [70]:
ctx = Context(agent)

In [71]:
respnse = await agent.run("Add trimmer to the cart")

ResponseError: model requires more system memory (15.9 GiB) than is available (12.6 GiB) (status code: 500)

In [36]:
respnse

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='Your trimmer has been successfully added to the cart. Let me know if you need anything else! 😊')]), structured_response=None, current_agent_name='Agent', raw={'model': 'qwen3:0.6b', 'created_at': '2025-10-21T10:31:59.354246162Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1144955232, 'load_duration': 54649128, 'prompt_eval_count': 309, 'prompt_eval_duration': 348181798, 'eval_count': 23, 'eval_duration': 726126303, 'message': Message(role='assistant', content='', thinking=None, images=None, tool_name=None, tool_calls=None), 'usage': {'prompt_tokens': 309, 'completion_tokens': 23, 'total_tokens': 332}}, tool_calls=[ToolCallResult(tool_name='addToCart', tool_kwargs={'item': 'trimmer'}, tool_id='addToCart', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text='Item added to the cart')], tool_name='addToCart', r

In [37]:
cart

['lipbam', 'trimmer']

In [60]:
response = await agent.run("My name is Aum", ctx=ctx)

In [61]:
response.response

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='My name is Aum. 😊')])

In [62]:
response = await agent.run("What is your name?", ctx=ctx)

In [63]:
response.response

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text='My name is Logan. 😊')])

In [66]:
response = await agent.run("What products do you have in your furniture store?", ctx=ctx)

In [68]:
response.response

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={'tool_calls': []}, blocks=[TextBlock(block_type='text', text="I don't have a specific product list. However, I can assist you with purchasing items if you have a product in mind. Can you tell me what product you're looking for? 😊")])